In [1]:
from abc import ABC
from abc import abstractmethod
import networkx as nx
import numpy as np
import progressbar
import random
import math

In [2]:
class LinkPrediction(ABC):
    def __init__(self, graph):
        self.graph = graph
        self.N = len(graph)
    
    def neighbors(self, v):
        neighbors_list = self.graph.neighbors(v)
        return list(neighbors_list)

    @abstractmethod
    def fit(self):
        raise NotImplementedError("Fit must be implemented")

In [3]:
class CommonNeighbors(LinkPrediction):
    def fit(self):
        scores = {}
        
        for u, v in nx.non_edges(self.graph):
            u_neighbors = set(self.neighbors(u))
            v_neighbors = set(self.neighbors(v))
            scores[(u, v)] = len(u_neighbors.intersection(v_neighbors))

        return scores

In [4]:
class Jaccard(LinkPrediction):
    def fit(self):
        scores = {}

        for u, v in nx.non_edges(self.graph):
            u_neighbors = set(self.neighbors(u))
            v_neighbors = set(self.neighbors(v))

            intersection = len(u_neighbors.intersection(v_neighbors))
            union = len(u_neighbors.union(v_neighbors))
            
            if union > 0:
                scores[(u, v)] = intersection / union
            else:
                scores[(u, v)] = 0.0

        return scores

In [5]:
class AdamicAdar(LinkPrediction):
    def fit(self):
        scores = {}

        for u, v in nx.non_edges(self.graph):
            u_neighbors = set(self.neighbors(u))
            v_neighbors = set(self.neighbors(v))
            common_neighbors = u_neighbors.intersection(v_neighbors)

            score = 0.0
            for w in common_neighbors:
                degree_w = len(list(self.neighbors(w)))
                if degree_w > 1:
                    score += 1 / math.log(degree_w)

            scores[(u, v)] = score

        return scores

In [6]:
def remove_random_edges(graph, fraction):
    edges = list(graph.edges())
    removed_edges = set(random.sample(edges, int(fraction * len(edges))))
    temp_graph = graph.copy()
    temp_graph.remove_edges_from(removed_edges)
    return temp_graph, removed_edges

def get_link_scores(predictor):
    scores = predictor.fit()
    return sorted(scores.items(), key=lambda x: x[1], reverse=True)

def evaluate(scores, removed_edges, k):
    top_pairs = set(pair for pair, _ in scores[:k])
    correct = len(removed_edges.intersection(top_pairs))
    precision = correct / k
    recall = correct / len(removed_edges)

    return precision, recall

In [7]:
# Read data
data_path = "../Data/fb100"

caltech = nx.read_gml(f"{data_path}/Caltech36.gml")
mit = nx.read_gml(f"{data_path}/MIT8.gml")
john_hopkins = nx.read_gml(f"{data_path}/Johns Hopkins55.gml")

In [8]:
fractions = [0.05, 0.1, 0.15, 0.2]
ks = [50, 100, 200, 300, 400]

In [9]:
def experiment(predictor, removed_edges): 
    scores = get_link_scores(predictor)
    for k in ks:
        precision, recall = evaluate(scores, removed_edges, k)

        print(f"k: {k}")
        print(f"precision: {precision}")
        print(f"recall: {recall}")

def common_neighbors_experiment(graph):
    for fraction in fractions:
        reduced_graph, removed_edges = remove_random_edges(graph, fraction)
        predictor = CommonNeighbors(reduced_graph)
        experiment(predictor, removed_edges)
        print("")

def jaccard_experiment(graph):
    for fraction in fractions:
        reduced_graph, removed_edges = remove_random_edges(graph, fraction)
        predictor = Jaccard(reduced_graph)
        experiment(predictor, removed_edges)
        print("")

def adamic_ada_experiment(graph):
    for fraction in fractions:
        reduced_graph, removed_edges = remove_random_edges(graph, fraction)
        predictor = AdamicAdar(reduced_graph)
        experiment(predictor, removed_edges)
        print("")

In [10]:
common_neighbors_experiment(caltech)

Graph with 769 nodes and 15824 edges
k: 50
precision: 0.14
recall: 0.008413461538461538
k: 100
precision: 0.16
recall: 0.019230769230769232
k: 200
precision: 0.15
recall: 0.036057692307692304
k: 300
precision: 0.15333333333333332
recall: 0.055288461538461536
k: 400
precision: 0.13
recall: 0.0625

Graph with 769 nodes and 14991 edges
k: 50
precision: 0.3
recall: 0.009009009009009009
k: 100
precision: 0.26
recall: 0.015615615615615615
k: 200
precision: 0.22
recall: 0.026426426426426425
k: 300
precision: 0.21333333333333335
recall: 0.03843843843843844
k: 400
precision: 0.195
recall: 0.04684684684684685

Graph with 769 nodes and 14158 edges
k: 50
precision: 0.28
recall: 0.005604483586869495
k: 100
precision: 0.28
recall: 0.01120896717373899
k: 200
precision: 0.275
recall: 0.022017614091273018
k: 300
precision: 0.27666666666666667
recall: 0.03322658126501201
k: 400
precision: 0.2525
recall: 0.04043234587670136

Graph with 769 nodes and 13325 edges
k: 50
precision: 0.38
recall: 0.00570399279

In [11]:
jaccard_experiment(caltech)

k: 50
precision: 0.12
recall: 0.007211538461538462
k: 100
precision: 0.09
recall: 0.010817307692307692
k: 200
precision: 0.075
recall: 0.018028846153846152
k: 300
precision: 0.08333333333333333
recall: 0.030048076923076924
k: 400
precision: 0.0825
recall: 0.039663461538461536

k: 50
precision: 0.22
recall: 0.006606606606606606
k: 100
precision: 0.3
recall: 0.018018018018018018
k: 200
precision: 0.24
recall: 0.02882882882882883
k: 300
precision: 0.20666666666666667
recall: 0.037237237237237236
k: 400
precision: 0.1825
recall: 0.04384384384384384

k: 50
precision: 0.24
recall: 0.004803843074459567
k: 100
precision: 0.25
recall: 0.010008006405124099
k: 200
precision: 0.235
recall: 0.018815052041633307
k: 300
precision: 0.21666666666666667
recall: 0.02602081665332266
k: 400
precision: 0.2125
recall: 0.034027221777421936

k: 50
precision: 0.2
recall: 0.003002101471029721
k: 100
precision: 0.22
recall: 0.006604623236265386
k: 200
precision: 0.23
recall: 0.013809666766736716
k: 300
precision:

In [12]:
adamic_ada_experiment(caltech)

k: 50
precision: 0.3
recall: 0.018028846153846152
k: 100
precision: 0.26
recall: 0.03125
k: 200
precision: 0.21
recall: 0.05048076923076923
k: 300
precision: 0.17666666666666667
recall: 0.06370192307692307
k: 400
precision: 0.1625
recall: 0.078125

k: 50
precision: 0.32
recall: 0.00960960960960961
k: 100
precision: 0.26
recall: 0.015615615615615615
k: 200
precision: 0.22
recall: 0.026426426426426425
k: 300
precision: 0.2
recall: 0.036036036036036036
k: 400
precision: 0.1825
recall: 0.04384384384384384

k: 50
precision: 0.28
recall: 0.005604483586869495
k: 100
precision: 0.32
recall: 0.012810248198558846
k: 200
precision: 0.3
recall: 0.02401921537229784
k: 300
precision: 0.2866666666666667
recall: 0.0344275420336269
k: 400
precision: 0.27
recall: 0.04323458767013611

k: 50
precision: 0.3
recall: 0.004503152206544582
k: 100
precision: 0.33
recall: 0.00990693485439808
k: 200
precision: 0.34
recall: 0.0204142900030021
k: 300
precision: 0.32
recall: 0.02882017412188532
k: 400
precision: 0.2